# **Lab 5: Python Databases, CSV Files and Map-Reduce**

In this lab, we will use Python for data processing using SQL databases. files and Map-Reduce.

Work **must** be done in your Colab notebook.  **Before you start working on your solution, save a copy to your Google Drive (as the original is read-only)**.   Answer question 1 and 2 directly in the notebook where indicated.  As you are working on the file, make sure to save as you  go.  When you are complete, save your notebook, download it and submit it via Canvas (You can download your notebook from the file menu ** File -> Download -> Download .ipynb **).  

With the Colab notebook, you can write and run your Python code directly in the bookbook.  To run your code, click on the play button and the left hand side of your code.   The output willl appear immediately below your code.

In [14]:
#This is a Python Comment
#Click the play button (on the left hand side of the code block) to run this code.
print("Hello!  My Python code runs!")

Hello!  My Python code runs!


**Importing a library that is not in Colaboratory**

When working with Python in Colab, you may find that a library in not available when you run yuor code.  In this case, you will need to import the library.  For this lab, you will need to install *mysql-python-connector*.  To import a library that's not in Colaboratory by default, you can use `!pip install` or `!apt-get install`.

Running this block is required anytime you connect/restart/reconnect to an instance. To run a code block, mouse over it and click the play button on left side.

You should see some output when you click the play button. Wait until the code block is finished running (this is indicated when the stop button goes away). The last couple lines of output should look something like the following:

```
Collecting mysql-connector-python
  Downloading mysql_connector_python-8.0.28-cp37-cp37m-manylinux1_x86_64.whl (37.6 MB)
     |████████████████████████████████| 37.6 MB 1.7 MB/s
Requirement already satisfied: protobuf>=3.0.0 in /usr/local/lib/python3.7/dist-packages (from mysql-connector-python) (3.17.3)
Requirement already satisfied: six>=1.9 in /usr/local/lib/python3.7/dist-packages (from protobuf>=3.0.0->mysql-connector-python) (1.15.0)
Installing collected packages: mysql-connector-python
Successfully installed mysql-connector-python-8.0.28
```


In [15]:
#Make sure to run this
!pip install mysql-connector-python

# **Objectives**

 * Use Python to connect to a MySQL relational database and execute SQL queries.
 * Output data to a CSV file.
 * Write a small Map-Reduce program for data analysis.

**Question #1 - Database Querying (3 marks)**

Write a Python program to query a MySQL database and print the query results.

**Details:**
*    Put a comment at the top of the question 1 Python notebook codeblock with your name and student number.
*    Make a connection to the MySQL database with this connection information:
```
 cnx = mysql.connector.connect(user='301', password='test', host='159.203.45.66', database='workson')
```
*    Write and execute a SQL query that returns the department number (dno), department name (dname), project number (pno), project name (pname), sum of hours worked on the project as well as the number of employees working on the project. Only include records where the budget is less than $200,000. Order by sum of hours worked ascending. (2 marks)
*    Print out your query results. Note that tabs ("\t") are used to line up the data in columns nicely. (0.5 mark)
*    Output your query results to a csv file called `l6q1.csv`.  The file will need a header (first row) followed the the results in a comma-separated format.  Use the [CSV writer](https://docs.python.org/3/library/csv.html).  Save the CSV file to the same location at your Python notebood (path location: `.`). (0.5 mark)

**WorksOn Relational Database Schema**
```
emp (eno CHAR(5), ename VARCHAR(30), bdate DATE, title CHAR(2),	salary DECIMAL(9,2), supereno CHAR(5), dno CHAR(5), PRIMARY KEY (eno))
dept (dno CHAR(5), dname VARCHAR(40), mgreno CHAR(5), PRIMARY KEY (dno))                      
          -- mgreno is the employee number of the manager of the department (may join with emp)
proj (pno CHAR(5), pname VARCHAR(40), budget DECIMAL(9,2), dno  CHAR(5), PRIMARY KEY (pno))   
          -- dno is the department that has the project (may join with dept)
workson (eno  CHAR(5), pno  CHAR(5), resp VARCHAR(20), hours SMALLINT, PRIMARY KEY (eno,pno))
          -- Use eno to join with emp.  Use pno to join with proj.
```
**Sample Output**
```
pno	 pname 		 dno 	 dname 		 hours 	 numemp
P1 	 Instruments 	 D1 	 Management 	 36 	 2
P2 	 DB Develop 	 D2 	 Consulting 	 48 	 3
```
The contents of the CSV file should be:
```
pno,pname,dno,dname,hours,numemp
P1,Instruments,D1,Management,36,2
P2,DB Develop,D2,Consulting,48,3
```
**Answer Question 1 in the following code block.**

In [28]:
#Answer question 1 here
#Name - Bhavya Bhagchandani
#Student# - 31715618

import mysql.connector
import csv

try:
  cnx = mysql.connector.connect(user='301', password='test', host='159.203.45.66', database='workson')
  cursor = cnx.cursor()
  query = ("select proj.pno, proj.pname, dept.dno, dept.dname, sum(workson.hours) as hours, count(workson.eno) as numemp "+
           "from workson "+
           "join proj on workson.pno = proj.pno "+
           "join dept on proj.dno = dept.dno "+
           "join emp on workson.eno = emp.eno "+
           "where proj.budget < %s "+
           "group by proj.pno, proj.pname, dept.dno, dept.dname "+
           "order by hours asc")

  cursor.execute(query,(200000, ))
  print("pno\t", "pname\t\t", "dno\t", "dname\t\t", "hours\t", "numemp\t")

  with open('16q1.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["pno", "pname", "dno", "dname", "hours", "numemp"])
    for (pno, pname, dno, dname, hours, numemp) in cursor:
      print(pno,"\t", pname,"\t", dno,"\t", dname,"\t", hours,"\t", numemp)
      spamwriter.writerow([pno, pname, dno, dname, hours, numemp])
  cursor.close()
except mysql.connector.Error as err:
  print(err)

finally:
  cnx.close()


pno	 pname		 dno	 dname		 hours	 numemp	
P1 	 Instruments 	 D1 	 Management 	 36 	 2
P2 	 DB Develop 	 D2 	 Consulting 	 48 	 3


**Question #2 - Database Querying (3 marks)**
*    Write and execute a SQL query that returns the manager employee number (mgreno), the amount of projects they manage and the sum of all their project's budgets. Only include records where the manager manages more than one project (2 marks)
*    Print out your query results. Note that tabs ("\t") are used to line up the data in columns nicely. (0.5 mark)
*    Output your query results to a csv file called `l6q2.csv`.  The file will need a header (first row) followed the the results in a comma-separated format.  Use the [CSV writer](https://docs.python.org/3/library/csv.html).  Save the CSV file to the same location at your Python notebood (path location: `.`). (0.5 mark)

In [103]:
#Answer question 2 here
#Name - Bhavya Bhagchandani
#Student# - 31715618
import mysql.connector
import csv

try:
  cnx = mysql.connector.connect(user='301', password='test', host='159.203.45.66', database='workson')
  cursor = cnx.cursor()
  query = ("select dept.mgreno, count(proj.pno) as numproj, sum(proj.budget) as amount "+
           "from workson "+
           "join proj on workson.pno = proj.pno "+
           "join dept on proj.dno = dept.dno "+
           "join emp on emp.eno = workson.eno "+
           "where workson.resp ='Manager'  "+
           "group by dept.mgreno "+
           "having numproj > 1")

  cursor.execute(query)
  print("mgreno\t", "numproj\t", "amount")

  with open('16q2.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["mgreno", "numproj", "amount"])
    for (mgreno, numproj, amount) in cursor:
      print(mgreno,"\t", numproj,"\t", amount)
      spamwriter.writerow([mgreno, numproj, amount])
  cursor.close()
except mysql.connector.Error as err:
  print(err)

finally:
  cnx.close()


mgreno	 numproj	 amount
E7 	 2 	 445000.00


**Question 3 - Map-Reduce (2 marks)**

Create a Python program that uses Map-Reduce to analyze a data set. ***You must use map, filter, and reduce functions***. Details:
 * Put a comment at the top of the question 2 Python codeblock with your name and student number.
 * Use a data set that consists of the numbers from 1 to 15. (0.5)
 * Apply a map function to the data set that will divide the value by 2 if it is even or multiply by 4 if it is odd. Print the result after the map function is applied. (0.5 marks)
 * Apply a filter function that will keep the value if it is between 3 and 30 inclusive. Print the result after the filter function is applied. (0.5 mark)
 * Apply a reduce function that will miltiply the two values if the first value is greater than the second otherwise it will add them. Print the result after reduce function is applied. (0.5 mark)

***Sample Output***
```
Result after map: [4, 1.0, 12, 2.0, 20, 3.0, 28, 4.0, 36, 5.0, 44, 6.0, 52, 7.0, 60]
Result after filter: [4, 12, 20, 3.0, 28, 4.0, 5.0, 6.0, 7.0]
Result after reduce: 2540160.0
```

**Answer Question 3 in the following code block.**

In [89]:
# Answer question 3 here
#Name - Bhavya Bhagchandani
#Student# - 31715618
import functools

# create dataset that contains number from 1 to 15, name the data list as "data"
data = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15);
print(data)

# complete the functions below

# Map function
# if even: divide by 2, if odd: multiply by 4
def modify(x):
    if (x % 2 == 0):
      return x / 2
    else:
      return x * 4

# Filter function
# keep the value between 3 and 30 inclusive
def myfilter(x):
    if (x >= 3 and x <= 30):
      return True
    return False


# Reduce function
# multiply 2 values if first number is greater than the second, add them if otherwise
def condsum(x, y):
    if (x>y):
      return x*y
    else:
      return x+y


result = list(map(modify, data))
print("Result after map:",result)

result = list(filter(myfilter, result))
print("Result after filter:",result)

result = functools.reduce(condsum, result)
print("Result after reduce:",result)

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
Result after map: [4, 1.0, 12, 2.0, 20, 3.0, 28, 4.0, 36, 5.0, 44, 6.0, 52, 7.0, 60]
Result after filter: [4, 12, 20, 3.0, 28, 4.0, 5.0, 6.0, 7.0]
Result after reduce: 2540160.0


**Question 4 - Map-Reduce (3 marks)**

Create a Python program that uses Map-Reduce to analyze a data set. ***You must use map, filter, and reduce functions***. Details:
 * Use the dataset in the code block below.
 * Apply a map function to calculate total sales per product(1 mark)
 * Apply a filter function that filters out products with total sales below $500. Print the result after the filter function is applied(1 mark)
 * Apply a reduce function that sums total sales across all products. Print the result after reduce function is applied(1 mark)

***Sample Output***
```
Mapped Sales (Total sales per product):
('Product A', 600)
('Product B', 600)
('Product C', 300)
('Product D', 750)
('Product E', 400)

Filtered Sales (Products with total sales >= $500):
('Product A', 600)
('Product B', 600)
('Product D', 750)

Total sales across all qualifying products: $1950
```

**Answer Question 4 in the following code block.**

In [94]:
from functools import reduce

# Sample dataset: a list of tuples (product_name, price, quantity_sold)
sales_data = [
    ('Product A', 120, 5),
    ('Product B', 200, 3),
    ('Product C', 150, 2),
    ('Product D', 50, 15),
    ('Product E', 400, 1),
]

# Step 1: Map - Define a function to calculate total sales per product
def modify(x):
  return (x[0], x[1]*x[2])

# Step 2: Filter - Define a function to filter out products with total sales below $500
def myfilter(x):
    if (x[1]>=500):
      return True
    return False

# Step 3: Reduce - Define a function to sum total sales across all products
def condsum(x, y):
  return x + y

result = list(map(modify, sales_data))
print("Result after map:",result)

result = list(filter(myfilter, result))
print("Result after filter:",result)

nums = []
for i in result:
  nums.append(i[1])

result = functools.reduce(condsum, nums)
print("Result after reduce:",result)


Result after map: [('Product A', 600), ('Product B', 600), ('Product C', 300), ('Product D', 750), ('Product E', 400)]
Result after filter: [('Product A', 600), ('Product B', 600), ('Product D', 750)]
Result after reduce: 1950


**Submission**

Submit your .ipynb (colab Python notebook) via Cavnas.  